# Installation

In [89]:

# !pip install -q openai
# !pip install -q gradio

# Imports

In [90]:
import openai
import gradio as gr
import itertools
import os
import requests
from urllib.parse import urlsplit, parse_qs
import random

# API Key 

openai.api_key = Replace this with your API key: https://beta.openai.com/docs/quickstart/add-your-api-key


In [91]:
openai.api_key = os.getenv("OPENAI_API_KEY") # Replace this with your API key: https://beta.openai.com/docs/quickstart/add-your-api-key

# OpenAI Chat

In [92]:
def openai_chat(prompt, history):
    start_sequence = "\nAI:"
    restart_sequence = "\nHuman: "
    prompt_engineering = """The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly. The AI assistant goal is to help the user find a mission statement that best aligns their beliefs in the DAO.  The DAO is called CityDAO wants to build the citys of the future.\n
    AI: I am an AI created by OpenAI. How can I help you define your mission statement?
    """

    history = [ (human_prompt.replace("<p>", "Human:").replace("</p>", "\n"), ai_prompt.replace("<p>", "AI:").replace("</p>", "\n" )) for human_prompt, ai_prompt in history]

    flat_history = list(itertools.chain(*history))


    prompt_history = prompt_engineering + ''.join(flat_history)

    

    prompt = prompt_history + ("Human: " + prompt + "\n")

    print("prompt:", repr(prompt))
    completions = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.9,
        max_tokens=150,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0.6,
        stop=[" Human:", " AI:"]
    )

    message = completions.choices[0].text
    return message.strip()

# AWS ZK Lambda Endpoint

In [93]:
# Create a Request Function that will submit id, identity, and message to the API
def createZKP_request(id, identity,group, attestation):
    # Create a dictionary of the parameters
    data = {
        "id": id,
        "identity": identity,
        "group": attestation,
        "attestation": "Missionbot", # Atttesation must be a string of 32 bytes or less
    }
    # Make a POST request to the AP
    response = requests.post("https://eczl8gqxk5.execute-api.eu-central-1.amazonaws.com/default/Semaphore-proof-function", json=data)
    return response.ok


# Gradio Interface Function

In [94]:
def chatbot(input, history=[]):
    output = openai_chat(input, history)
    history.append((input, output))
    return history, history

In [95]:
def echo(input, history = []):
  history.append((input, "Hello World"))
  return history, history

# Launch Interface

In [97]:
# Gradio Interface using gradio blocks 
# Retrive Gradio URL parameter from the URL

with gr.Blocks() as demo:

  history = gr.State([])
  
  with gr.Column(visible=True) as inital_column:
    gr.Markdown(
      """
      # Fellow Community Member!
      I am missionBot! I am here to know your mission. Please start in the chat so I can get a sense of your mission, 
      and when you are done just submit your mission!
      """)
    outputDialogue = gr.Chatbot( )
    inputText = gr.Textbox()

    greet_btn = gr.Button(value="Submit my Mission", visible=False)
  
  with gr.Column(visible=False) as final_column:
  
    gr.Markdown(
      """
      Thank you for your mission! Your mission has been sent to the DAO for review.
      To start a new mission please refresh the page!
      """
    )
    markdownID = gr.Markdown()
    
    
  def btn_click(input, history, request: gr.Request):
    print("Request headers dictionary:", request.headers)
    referrer = request.headers.get("Referer")
    query = urlsplit(referrer).query
    params = parse_qs(query)
    print("Params:", params)
    # if params id exists enter if statement
    if "id" in params:
       response = createZKP_request(int(params["id"]), "None", "None", str(history))
       # Update the state and rerender the interface
       return {
                inital_column: gr.update(visible=False),
                final_column: gr.update(visible=True),
                markdownID: gr.update(value=f"Your mission number is: {params['id']}")
            } 
    else:
      random_id = random.randrange(1000,8100)
      response = createZKP_request(int(random_id), "None", "None", str(history))

      # Update the state and rerender the interface
      return {
              inital_column: gr.update(visible=False),
              final_column: gr.update(visible=True),
              markdownID: gr.update(value=f"Your mission Number is: {random_id}")
          } 
  
  
  inputText.submit(fn=chatbot, inputs=[inputText, history], outputs=[outputDialogue, history])

  greet_btn.click(fn=btn_click, inputs=[inputText,history], outputs=[inital_column, final_column, markdownID])
  outputDialogue.change(fn=lambda value: gr.update(visible=True) , inputs=outputDialogue, outputs=greet_btn)


demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


prompt: 'The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly. The AI assistant goal is to help the user find a mission statement that best aligns their beliefs in the DAO.  The DAO is called CityDAO wants to build the citys of the future.\n\n    AI: I am an AI created by OpenAI. How can I help you define your mission statement?\n    Human: can you bring the heat!\n'
Request headers dictionary: Headers({'host': '127.0.0.1:7861', 'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0', 'accept': '*/*', 'accept-language': 'en-US,en;q=0.5', 'accept-encoding': 'gzip, deflate, br', 'referer': 'http://127.0.0.1:7861/', 'content-type': 'application/json', 'content-length': '83', 'origin': 'http://127.0.0.1:7861', 'connection': 'keep-alive', 'sec-fetch-dest': 'empty', 'sec-fetch-mode': 'cors', 'sec-fetch-site': 'same-origin', 'sec-gpc': '1'})
Params: {}
